In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as mpl

from collections import defaultdict
from functools import reduce
from path import Path
from pprint import pprint
from statsmodels.tsa.stattools import adfuller

%matplotlib inline
mpl.style.use('ggplot')
mpl.rcParams['figure.figsize'] = 16,6

A few interesting notes from this chapter:

**Marcos' first law of backtesting:**

**Backtesting is not a research tool. Feature importance is.**


Once we have found what features are important, we can learn more by conducting a number of experiments.

- Are these features important all the time, or only in some specific environments?
- What triggers a change in importance over time?
- Can these regime switches be predicted?
- Are those important features also relevant to other related financial instruments?
- Ahe they relevant to other asset classes?
- What are the most relevant features across all financial instruments?
- What is the subset of features with the  highest rank correlation across the entire investment universe?



In [2]:
from itertools import product
from multiprocess import mpPandasObj
from cv import PurgedKFold, cvScore
from feature_imp import featImpMDI, featImpMDA, auxFeatImpSFI, getTestData

# Code from Chapter 8

def featImportance(trnsX, cont, n_estimators=1000, cv=10, max_samples=1.0, numThreads=12, pctEmbargo=0, scoring='accuracy',
                   method='SFI', minWLeaf=0.0, **kwargs):
    # feature importance from a random forest
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    n_jobs = -1 if numThreads > 1 else 1
    # 1) prepare classifier, cv. max_features=1 to prevent masking
    clf = DecisionTreeClassifier(criterion='entropy', max_features=1, class_weight='balanced', min_weight_fraction_leaf=minWLeaf)
    clf = BaggingClassifier(base_estimator=clf, n_estimators=n_estimators, max_features=1, max_samples=max_samples, oob_score=True, n_jobs=n_jobs)
    fit = clf.fit(X=trnsX, y=cont['bin'], sample_weight=cont['w'].values)
    oob = fit.oob_score_
    if method == 'MDI':
        imp = featImpMDI(fit, featNames=trnsX.columns, max_features=max_features)
        oos = cvScore(clf, X=trnsX, y=cont['bin'], cv=cv, sample_weight=cont['w'], t1=cont['t1'], pctEmbargo=pctEmbargo, scoring=scoring).mean()
    elif method == 'MDA':
        imp, oos = featImpMDA(clf, X=trnsX, y=cont['bin'], cv=cv, sample_weight=cont['w'], t1=cont['t1'], pctEmbargo=pctEmbargo, scoring=scoring)
    elif method == 'SFI':
        cvGen = PurgedKFold(n_splits=cv, t1=cont['t1'], pctEmbargo=pctEmbargo)
        oos = cvScore(clf, X=trnsX, y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cvGen=cvGen).mean()
        clf.n_jobs = 1 # parallelize auxFeatImpSFi rather than clf
        imp = mpPandasObj(auxFeatImpSFI, ('featNames', trnsX.columns), numThreads, clf=clf, trnsX=trnsX, cont=cont, scoring=scoring, cvGen=cvGen)

    return imp, oob, oos

def testFunc(n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10):
    # test the importance of the feat importance functions on artificial data
    # Nr noise features = n_features - n_informative - n_redundant
    trnsX, cont = getTestData(n_features, n_informative, n_redundant, n_samples)
    return testDataFunc(trnsX, cont, n_estimators, cv)

def testDataFunc(trnsX, cont, n_estimators=1000, cv=10, tag='testFunc', methods=['MDI', 'MDA', 'SFI']):
    dict0 = {'minWLeaf': [0.0], 'scoring': ['accuracy'], 'method': methods, 'max_samples':[1.0]}
    jobs, out = (dict(zip(dict0, i)) for i in product(*dict0.values())), []
    kwargs = {'pathOut': './testFunc/', 'n_estimators': n_estimators, 'tag': tag, 'cv': cv}
    for job in jobs:
        job['simNum'] = job['method'] + '_' + job['scoring'] + '_' + '%.2f' % job['minWLeaf'] + '_' + str(job['max_samples'])
        print(job['simNum'])
        kwargs.update(job)
        imp, oob, oos = featImportance(trnsX, cont=cont, **kwargs)
        plotFeatImportance(imp=imp, oob=oob, oos=oos, **kwargs)
        df0 = imp[['mean']] / imp['mean'].abs().sum()
        df0['type'] = [i[0] for i in df0.index]
        df0 = df0.groupby('type')['mean'].sum().to_dict()
        df0.update({'oob': oob, 'oos': oos})
        df0.update(job)
        out.append(df0)
    out = pd.DataFrame(out).sort_values(['method', 'scoring', 'minWLeaf', 'max_samples'])
    out = out[['method', 'scoring', 'minWLeaf', 'max_samples', 'I', 'R', 'N', 'oob', 'oos']]
    out.to_csv(kwargs['pathOut'] + 'stats.csv')
    return

def plotFeatImportance(pathOut, imp, oob, oos, method, tag=0, simNum=0, **kwargs):
    # plot mean imp bars with std
    mpl.figure(figsize=(10, imp.shape[0] / 5.0))
    imp = imp.sort_values('mean', ascending=True)
    ax = imp['mean'].plot(kind='barh', color='b', alpha=0.25, xerr=imp['std'], error_kw={'ecolor': 'r'})
    
    if method == 'MDI':
        mpl.xlim([0, imp.sum(axis=1).max()])
        mpl.axvline(1.0 / imp.shape[0], linewidth=1, color='r', linestyle='dotted')
    ax.get_yaxis().set_visible(False)
    for i, j in zip(ax.patches, imp.index):
        ax.text(i.get_width() / 2, i.get_y() + i.get_height() / 2, j, ha='center', va='center', color='black')
    mpl.title('tag=' + tag + ' | simNum=' + str(simNum) + ' | oob=' + str(round(oob, 4)) + ' | oos=' + str(round(oos, 4)))
    mpl.savefig(pathOut + 'featImportance_' + str(simNum) +'_tag_' + tag + '.png', dpi=100)
    mpl.clf()
    mpl.close()
    return

def get_eVec(dot, varThres):
    # compute eVec from dot prod matrix, reduce dimension
    eVal, eVec = np.linalg.eigh(dot)
    idx = eVal.argsort()[::-1]
    eVal, eVec = eVal[idx], eVec[:, idx]
    
    eVal = pd.Series(eVal, index=['PC_' + str(i+1) for i in range(eVal.shape[0])])
    eVec = pd.DataFrame(eVec, index=dot.index, columns=eVal.index)
    eVec = eVec.loc[:, eVal.index]
    
    cumVar = eVal.cumsum() / eVal.sum()
    dim = cumVar.values.searchsorted(varThres)
    eVal, eVec = eVal.iloc[:dim + 1], eVec.iloc[:, :dim + 1]
    return eVal, eVec

def orthoFeats(dfX, varThres=0.95):
    dfZ = dfX.sub(dfX.mean(), axis=1).div(dfX.std(), axis=1)
    dot = pd.DataFrame(np.dot(dfZ.T, dfZ), index=dfX.columns, columns=dfX.columns)
    eVal, eVec = get_eVec(dot, varThres)
    dfP = np.dot(dfZ, eVec)
    return dfP

# testFunc(n_features=20, n_informative=5, n_redundant=5, n_estimators=100, n_samples=1000, cv=10) 

# 8.1a

Using the code presented in Section 8.6

Generate a dataset $(X, y)$

In [9]:
trnsX, cont = getTestData(n_features=12, n_informative=4, n_redundant=4, n_samples=10000,)
trnsX.head()

,I_0,I_1,I_2,I_3,R_0,R_1,R_2,R_3,N_0,N_1,N_2,N_3
2019-11-07 22:48:33.686192,-2.689873,0.029924,-3.018511,2.235000,5.249226,1.497503,-4.259974,0.535160,0.497745,0.834478,-0.678012,1.122451
2019-11-07 22:49:33.686192,-2.101359,-1.472394,-0.813956,1.586275,2.211585,-0.217206,-3.401780,-0.442523,-1.177745,0.068846,-1.687707,1.116839
2019-11-07 22:50:33.686192,-1.999457,-2.212843,-0.398948,1.137350,1.089805,-1.122141,-3.636591,-0.737204,-0.242808,-0.113313,-0.768828,-0.167736
2019-11-07 22:51:33.686192,-3.549063,-2.882772,-0.937247,1.138791,1.772129,-2.087764,-5.586152,-0.189952,-1.212815,1.421950,0.729629,0.847081
2019-11-07 22:52:33.686192,-0.988358,-0.557092,-1.176953,1.439323,2.270765,0.767248,-2.213849,-0.484104,-0.731081,0.423829,2.072763,0.050671


# 8.1b

Using the code presented in Section 8.6

Apply a PCA transformation on X, which we denote $\dot X$.

In [10]:
Xdot = pd.DataFrame(orthoFeats(trnsX, 1.01))
Xdot.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-2.107580,-1.571988,-3.411484,0.174110,1.646305,0.020611,0.025204,-0.080748,-7.771561e-16,-8.604228e-16,6.661338e-16,-9.992007e-16
1,-1.111479,-0.081332,-1.868084,-1.642531,1.173064,-0.263628,-0.965279,1.162019,-5.551115e-16,-5.481726e-16,7.771561e-16,-2.220446e-16
2,-0.780078,0.752443,-1.502150,-0.493071,0.063027,0.487832,-0.433514,1.375909,-4.440892e-16,-5.100087e-16,5.828671e-16,2.775558e-16
3,-0.866288,1.392241,-2.991825,0.569623,0.373976,-1.732524,-0.786618,1.720167,-8.881784e-16,-1.665335e-16,3.330669e-16,0.000000e+00
4,-1.169347,-0.742145,-1.238146,0.973626,-1.020943,-1.684008,0.340825,0.260088,-5.551115e-16,-8.187895e-16,4.996004e-16,-6.661338e-16


# 8.1c

Using the code presented in Section 8.6

Compute MDI, MDA, and SFI feature importance on $(\dot X, y)$, where the base estimator is a RF.

In [11]:
Xdot = Xdot.set_index(trnsX.index)
Xdot.columns = trnsX.columns
# testDataFunc(Xdot, cont)


# 8.1d

Using the code presented in Section 8.6

Do the three methods agree on what features are important? Why?

![title](testFunc/81c_mdi.png)
![title](testFunc/81c_mda.png)
![title](testFunc/81c_sfi.png)

**A: While the formatting is slightly screwy on MDI, both MDI and MDA manage to filter out the informative features, though MDI with a higher degree of confidence. SFI strangely puts noise at the top of the list -- albeit with large confidence bands, that if taken into account, also have it with the informative features at the top.**

# 8.2a

From exercise 1, generate a new dataset $(\ddot X, y)$, where $\ddot X$ is a feature union of $X$ and $\dot X$.

Compute MDI, MDA, and SFI feature importance on $(\ddot X, y)$, where the base estimator is a RF.

In [12]:
Xdotdot = pd.concat([trnsX, Xdot.add_prefix('Xdot_')], axis=1)
# testDataFunc(Xdotdot, cont)

# 8.2b

From exercise 1, generate a new dataset $(\ddot X, y)$, where $\ddot X$ is a feature union of $X$ and $\dot X$.

Do the three methods agree on what features are important? Why?

![title](testFunc/82b_mdi.png)
![title](testFunc/82b_mda.png)
![title](testFunc/82b_sfi.png)

**A: If filtering out results with large confidence bands (N in SFI), then all methods tend rank the original non-transformed informative and redundant features over their PCA-transformed counterparts and those over any noisy ones.**

# 8.3a

Take the results from exercise 2: 

Drop the most important features according to each method, resulting in a features matrix $\dddot X$.

In [31]:
most_important_features = ['Xdot_I_0', 'Xdot_N_0', 'Xdot_N_1', 'I_2', 'R_2']
Xdotdotdot = Xdotdot.loc[:, ~Xdotdot.columns.isin(most_important_features)]

# 8.3b

Take the results from exercise 2: 

Compute MDI, MDA, and SFI feature importance on $(\dddot X, y)$, where the base estimator is a RF.

In [ ]:
# testDataFunc(Xdotdotdot, cont)

![title](testFunc/83b_mdi.png)
![title](testFunc/83b_mda.png)
![title](testFunc/83b_sfi.png)

# 8.3c

Take the results from exercise 2: 

Do you appreciate significant changes in the rankings of important features, relative to the results from exercise 2?

**A: MDI seems unperturbed.**

**The mean of MDA has dramatically shifted, and it is no longer assigning negative importance to a few informative and redundant features.**

**Removing the 2 noisy features at the top from SFI has cleared up the picture a lot, without seemingly affecting the rest.**

# 8.4a

Using the code presented in Section 8.6:

Generate a dataset $(X, y)$ of 1E6 observations, where 5 features are informative, 5 are redundant and 10 are noise.

In [3]:
trnsX, cont = getTestData(n_features=20, n_informative=5, n_redundant=5, n_samples=int(1e5))

# 8.4b

Using the code presented in Section 8.6:

Split $(X, y)$ into 10 datasets, each of 1E5 observations.

**A: Implemented in thenext answer.**

# 8.4c

Using the code presented in Section 8.6:

Compute the parallelized feature importance on each of the 10 datasets.

In [3]:
def mean_of_dfs(dfs):
    # I'm confident there's a nicer way to do this...
    return reduce(lambda left, right: left.add(right), dfs) / len(dfs)
    

def testDataFuncN(trnsX, cont, n_estimators=1000, cv=10, n_chunks=10, tag='testFunc'):
    dict0 = {'minWLeaf': [0.0], 'scoring': ['accuracy'], 'method': ['MDI'], 'max_samples':[1.0]} # 'SFI' 'MDA',
    jobs, out = (dict(zip(dict0, i)) for i in product(*dict0.values())), []
    kwargs = {'pathOut': './testFunc/', 'n_estimators': n_estimators, 'tag':tag, 'cv':cv}
    for job in jobs:
        job['simNum'] = job['method'] + '_' + job['scoring'] + '_' + '%.2f' % job['minWLeaf'] + '_' + str(job['max_samples'])
        print(job['simNum'])
        kwargs.update(job)
        imps, oobs, ooss = [], [], []
        for i, chunk in enumerate(np.array_split(trnsX.index, n_chunks)):
            trns_chunk = trnsX[trnsX.index.isin(chunk)]
            cont_chunk = cont[cont.index.isin(chunk)]
            imp, oob, oos = featImportance(trns_chunk, cont=cont_chunk, **kwargs)
            imps.append(imp)
            oobs.append(oob)
            ooss.append(oos)
        imp, oob, oos = mean_of_dfs(imps), pd.Series(oobs).mean(), pd.Series(ooss).mean()
        plotFeatImportance(imp=imp, oob=oob, oos=oos, **kwargs)
        df0 = imp[['mean']] / imp['mean'].abs().sum()
        df0['type'] = [i[0] for i in df0.index]
        df0 = df0.groupby('type')['mean'].sum().to_dict()
        df0.update({'oob': oob, 'oos': oos})
        df0.update(job)
        out.append(df0)
    out = pd.DataFrame(out).sort_values(['method', 'scoring', 'minWLeaf', 'max_samples'])
    out = out[['method', 'scoring', 'minWLeaf', 'max_samples', 'I', 'R', 'N', 'oob', 'oos']]
    out.to_csv(kwargs['pathOut'] + 'stats.csv')
    return

# testDataFuncN(trnsX, cont, n_chunks=10)

#### Parallelized feature importance:

![title](testFunc/84d_mdi_10chunks.png)

# 8.4d

Using the code presented in Section 8.6:

Compute the stacked feature importance on the combined dataset $(X, y)$.

In [5]:
testDataFuncN(trnsX, cont, n_chunks=1)

MDI_accuracy_0.00_1.0
MDA_accuracy_0.00_1.0


#### Stacked feature importance:

![title](testFunc/84d_mdi_1chunk.png)


# 8.4e 

Using the code presented in Section 8.6:

What causes the discrepancy between the two? Which one is more reliable?

**A: In terms of ranking informative and redundant features above noisy ones, both methods achieve the same result, while the more computationally intensive (stacked) does so by a much wider margin.**

# 8.5

Repeat all MDI calculations from exercises 1-4, but this time allow for masking effects. That means, do not set `max_features=int(1)` in Snippet 8.2. How do results differ as a consequence of this change? Why?

In [3]:
# The code above and in feature_imp.py was temporarily modified to incorporate these and then used to generate the figures below
trnsX, cont = getTestData(n_features=12, n_informative=4, n_redundant=4, n_samples=10000,)
Xdot = pd.DataFrame(orthoFeats(trnsX, 1.01))
Xdot = Xdot.set_index(trnsX.index)
Xdot.columns = trnsX.columns
# testDataFunc(Xdot, cont, tag='85_1')
Xdotdot = pd.concat([trnsX, Xdot.add_prefix('Xdot_')], axis=1)
# testDataFunc(Xdotdot, cont, tag='85_2')


MDI_accuracy_0.00_1.0
MDI_accuracy_0.00_1.0


![title](testFunc/85_1.png)
![title](testFunc/85_2.png)


**A: There is little change for the PCA-transformed features, while MDI seems to perform a lot better on the union of transformed and non-transformed features when allowing for masking effects.**

In [5]:
most_important_features = ['Xdot_I_0', 'Xdot_N_0', 'Xdot_N_1', 'I_2', 'R_2', 'R_0']
Xdotdotdot = Xdotdot.loc[:, ~Xdotdot.columns.isin(most_important_features)]
testDataFunc(Xdotdotdot, cont, tag='85_3')

MDI_accuracy_0.00_1.0


![title](testFunc/85_3.png)


In [4]:
trnsX, cont = getTestData(n_features=20, n_informative=5, n_redundant=5, n_samples=int(1e5))
testDataFuncN(trnsX, cont, n_chunks=10, tag='85_4')
testDataFuncN(trnsX, cont, n_chunks=1, tag='85_5')

# way faster to do in chunks (10x likely)

MDI_accuracy_0.00_1.0
MDI_accuracy_0.00_1.0


#### Parallelized feature importance:
![title](testFunc/85_4.png)

#### Stacked feature importance:
![title](testFunc/85_5.png)


**A: It appears parallelized feature importance completely falls apart when allowing for masking effects, while the stacked method remains solid.**